# Simple NN with MINST images

Imports:

In [ ]:
import torch
import numpy as np
from torch import nn, tensor, optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch.nn.functional import normalize
from time import perf_counter

Lets download a data automatically. On downloaded data we do some preprocessing. Tensors are used by pytorch so we need to put downloaded data into tensors, and later we will normalize the data. For more on normalization see here: https://cs231n.github.io/neural-networks-2/

Downloaded data is split into train and test sets. Test set will be used at the end to see how well model is doing.


In [ ]:
import torch.nn.functional as F
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

dataset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
print(len(dataset))
trainset, testset = random_split(dataset, [50000, 10000])
print(len(trainset), len(testset))
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)


Lets define a simple NN network:

In [ ]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

Let's train it. SGD is used as a standard option.

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)
epochs = 15

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

After training we can check how well model behaves on test data. Most likely this is not the best piece of code, but at least it works:

In [ ]:
testloader = DataLoader(testset, batch_size=1)

with torch.no_grad():
    model.eval()

    correct = 0
    total = cnt = 1000
    for images, labels in testloader:
        cnt -= 1
        images = images.view(images.shape[0], -1)
        
        yhat = model(images)
        index = np.argmax(yhat[0])
        
        if (index == labels):
            correct += 1
        else:
            print("Index != label", index, labels)
        if cnt <= 0:
            break
            
print(correct / float(total))
        